In [1]:
import numpy as np
import pickle

In [ ]:
def prepare_features(df, training_features, variation="nominal", add_year=False):
    #global training_features
    if add_year:
        features = training_features + ["year"]
    else:
        features = training_features
    features_var = []
    #print(features)
    for trf in features:
        if f"{trf}_{variation}" in df.columns:
            features_var.append(f"{trf}_{variation}")
        elif trf in df.columns:
            features_var.append(trf)
        else:
            print(f"Variable {trf} not found in training dataframe!")
    return features_var

    

def evaluate_bdt(df, variation, model, parameters, score_name):
    training_features = ['dimuon_cos_theta_cs', 'dimuon_dEta', 'dimuon_dPhi', 'dimuon_dR', 'dimuon_eta', 'dimuon_phi', 'dimuon_phi_cs', 'dimuon_pt', 'dimuon_pt_log', 'jet1_eta_nominal', 'jet1_phi_nominal', 'jet1_pt_nominal', 'jet1_qgl_nominal', 'jet2_eta_nominal', 'jet2_phi_nominal', 'jet2_pt_nominal', 'jet2_qgl_nominal', 'jj_dEta_nominal', 'jj_dPhi_nominal', 'jj_eta_nominal', 'jj_mass_nominal', 'jj_mass_log_nominal', 'jj_phi_nominal', 'jj_pt_nominal', 'll_zstar_log_nominal', 'mmj1_dEta_nominal', 'mmj1_dPhi_nominal', 'mmj2_dEta_nominal', 'mmj2_dPhi_nominal', 'mmj_min_dEta_nominal', 'mmj_min_dPhi_nominal', 'mmjj_eta_nominal', 'mmjj_mass_nominal', 'mmjj_phi_nominal', 'mmjj_pt_nominal', 'mu1_eta', 'mu1_iso', 'mu1_phi', 'mu1_pt_over_mass', 'mu2_eta', 'mu2_iso', 'mu2_phi', 'mu2_pt_over_mass', 'zeppenfeld_nominal']
    df = df[training_features].compute()

    # df.loc[:,'mu1_pt_over_mass'] = np.divide(df['mu1_pt'], df['dimuon_mass'])
    # df.loc[:,'mu2_pt_over_mass'] = np.divide(df['mu2_pt'], df['dimuon_mass'])

    # df['njets_nominal'].fillna(0)
    df['mu1_pt_over_mass'] = df['mu1_pt']/df['dimuon_mass']
    df['mu2_pt_over_mass'] = df['mu2_pt']/df['dimuon_mass']
    df['njets'] = ak.fill_none(df['njets_nominal'], value=0)

    #df[df['njets_nominal']<2]['jj_dPhi_nominal'] = -1
    
    df.fillna(-99.0)
    df.loc[:,'mmj_min_dEta_nominal'] = df["mmj2_dEta_nominal"]
    df.loc[:,'mmj_min_dPhi_nominal'] = df["mmj2_dPhi_nominal"]
    features = prepare_features(df,training_features, variation=variation, add_year=False)
    #model = f"{model}_{parameters['years'][0]}"
    score_name = f"score_{model}_{variation}"

    # df.loc[:, score_name] = 0
    score_total = ak.ones_like(df['mu1_pt']) * -999.0
    
    nfolds = 4
    columns_print = ['njets_nominal','jj_dPhi_nominal','jj_mass_log_nominal', 'jj_phi_nominal', 'jj_pt_nominal', 'll_zstar_log_nominal', 'mmj1_dEta_nominal','jet2_pt_nominal']
    columns2 = ['mmj1_dEta_nominal', 'mmj1_dPhi_nominal', 'mmj2_dEta_nominal', 'mmj2_dPhi_nominal', 'mmj_min_dEta_nominal', 'mmj_min_dPhi_nominal']

    
    for i in range(nfolds):
        # think eval_folds is the list of test dataset chunks that each bdt is trained to evaluate
        eval_folds = [(i + f) % nfolds for f in [3]]

        eval_filter = df.event.mod(nfolds).isin(eval_folds)
        scalers_path = f"{parameters['models_path']}/{model}/scalers_{model}_{i}.npy"
        scalers = np.load(scalers_path, allow_pickle=True)
        model_path = f"{parameters['models_path']}/{model}/{model}_{i}.pkl"

        bdt_model = pickle.load(open(model_path, "rb"))
        df_i = df[eval_filter]
        if df_i.shape[0] == 0:
            continue
        df_i.loc[df_i.region != "h-peak", "dimuon_mass"] = 125.0

        df_i = (df_i[features] - scalers[0]) / scalers[1]
        if len(df_i) > 0:
            print(f"model: {model}")
            prediction = np.array(
                bdt_model.predict_proba(df_i.values)[:, 1]
            ).ravel()
            
            # df.loc[eval_filter, score_name] = prediction  # np.arctanh((prediction))
            df[score_name] = ak.where(eval_filter, prediction, df[score_name])


In [2]:
model_path = "/depot/cms/hmm/vscheure/data/trained_models/BDTv12_2018/BDTv12_2018_0.pkl"
bdt_model = pickle.load(open(model_path, "rb"))

In [ ]:
bdt_model

In [4]:
type(bdt_model)

xgboost.sklearn.XGBClassifier

In [5]:
bdt_model.save_model("model-0.json")

# dump model with feature map
bdt_model.get_booster().dump_model("dump.nice.txt")

In [4]:
# ! pip list
